In [2]:
import sys
import logging
import argparse

from fedtraj.config import Config
from fedtraj.utils import tool_funcs


args = {
    "dataset": "porto",
    "cell_size": 100,
    "test_type": "distort",
    "method": "fcl",
}

Config.update(args)

logging.basicConfig(
    level=logging.DEBUG if Config.debug else logging.INFO,
    format="[%(filename)s:%(lineno)s %(funcName)s()] -> %(message)s",
    handlers=[
        logging.FileHandler(
            Config.root_dir + "/exp/log/" + tool_funcs.log_file_name(), mode="w"
        ),
        logging.StreamHandler(),
    ],
)

logging.info("python " + " ".join(sys.argv))
logging.info("=================================")
logging.info(Config.to_str())
logging.info("=================================")

if Config.method in ["fcl", "fedavg"]:
    from fedtraj.model.trainer import FedTrajCLTrainer

    trainer_class = FedTrajCLTrainer
else:
    from fedtraj.model.trainer import TrajCLTrainer

    trainer_class = TrajCLTrainer

if Config.method == "fcl":
    from fedtraj.model.trajcl import TrajCL_with_Buffer as TrajCL
else:
    from fedtraj.model.trajcl import TrajCL

trajcl = trainer_class(Config.trajcl_aug1, Config.trajcl_aug2, model_class=TrajCL)
# trajcl.load_checkpoint()
trajcl.train()
trajcl.test()
# lcss_test()
# trajcl.knn_test('discrete_frechet')

[3852832567.py:29 <module>()] -> python d:\installation\conda\envs\py310\lib\site-packages\ipykernel_launcher.py --f=c:\Users\A1495\AppData\Roaming\jupyter\runtime\kernel-v3cd2a12560736939dcf70611eb1f4ae81621fa281.json
[3852832567.py:30 <module>()] -> =================================
[3852832567.py:31 <module>()] -> debug = True
dumpfile_uniqueid = 
seed = 2000
device = cuda:0
root_dir = d:\codes\TrajMM\fedtraj
checkpoint_dir = d:\codes\TrajMM\fedtraj/exp/snapshots
dataset = porto
dataset_prefix = porto_20200
dataset_file = d:\codes\TrajMM\fedtraj/data/porto_20200
dataset_cell_file = d:\codes\TrajMM\fedtraj/data/porto_20200_cell100_cellspace.pkl
dataset_embs_file = d:\codes\TrajMM\fedtraj/data/porto_20200_cell100_embdim32_embs.pkl
method = fcl
min_lon = -8.7005
min_lat = 41.1001
max_lon = -8.5192
max_lat = 41.2086
max_traj_len = 200
min_traj_len = 5
cell_size = 100
cellspace_buffer = 50
trajcl_batch_size = 128
cell_embedding_dim = 32
seq_embedding_dim = 32
moco_proj_dim = 16
moco_nque

d:\codes\TrajMM\fedtraj\utils\trajclus.py:19: RuntimeWarning: invalid value encountered in double_scalars
  projection = line[0] + (dot_product / line_length_squared) * line_vector
163it [00:00, 577.12it/s]
164it [00:00, 602.00it/s]
164it [00:01, 148.82it/s]
164it [00:00, 236.54it/s]
164it [00:15, 10.66it/s]
[utils.py:128 get_federated_segments()] -> [Split dataset and convert to segments] done
164it [00:20,  7.84it/s]
d:\installation\conda\envs\py310\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
d:\installation\conda\envs\py310\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.w

In [4]:
import torch
from torch.nn.utils.rnn import pad_sequence
from fedtraj.utils.traj import *
from fedtraj.model.trainer import TrajCLTrainer

trajcl.load_checkpoint()

if isinstance(trajcl, TrajCLTrainer):
    model = trajcl.model
else:
    model = trajcl.server.model

cellspace = trajcl.cellspace
cellspace_emb = trajcl.embs


def get_embedding(single_traj):
    return batch_get_embedding([single_traj])


def batch_get_embedding(batch_traj):
    # 将 single_traj 封装为列表并直接计算 trajs2_cell 和 trajs2_p
    trajs2_cell, trajs2_p = zip(*[merc2cell2(t, cellspace) for t in batch_traj])

    # 生成并填充 trajs2_emb_p
    trajs2_emb_p = pad_sequence(
        [torch.tensor(generate_spatial_features(t, cellspace)) for t in trajs2_p],
        batch_first=False,
    ).to(Config.device)

    # 生成并填充 trajs2_emb_cell
    trajs2_emb_cell = pad_sequence(
        [cellspace_emb[list(t)] for t in trajs2_cell], batch_first=False
    ).to(Config.device)

    # 计算 trajs2_len
    trajs2_len = torch.tensor(
        [len(t) for t in trajs2_cell], dtype=torch.long, device=Config.device
    )

    # print(single_traj, trajs2_cell, trajs2_p, trajs2_len)

    # 调用模型进行解释并返回结果
    return model.interpret(trajs2_emb_cell, trajs2_emb_p, trajs2_len)

In [5]:
from fedtraj.utils.tool_funcs import lonlat2meters


def convert_lonlat_traj(trajectory):
    merc_traj = [lonlat2meters(point[0], point[1]) for point in trajectory]
    return merc_traj


test_traj = [
    (-8.574678, 41.151951),
    (-8.574705, 41.151942),
    (-8.574696, 41.151933),
    (-8.574723, 41.151933),
    (-8.574714, 41.151924),
    (-8.574714, 41.151924),
    (-8.575164, 41.150934),
    (-8.577135, 41.150232),
    (-8.57853, 41.148639),
    (-8.579745, 41.147316),
]

get_embedding(convert_lonlat_traj(test_traj)).shape

torch.Size([1, 32])

In [ ]:
from easydict import Easydict


class Segment(EasyDict):
    def __init__(self, segment_id, points, emb):
        super().__init__()
        self.id = segment_id
        self.points = points
        self.emb = emb

In [6]:
import pandas as pd
import os
from fedtraj.utils.data_loader import TrajDataset
from fedtraj.model.trainer.utils import *
from tqdm import tqdm

trajs = pd.read_pickle(Config.dataset_file)
traj_dataset = get_federated_segments(TrajDataset(trajs), fed_num=Config.cls_num)


[utils.py:125 get_federated_segments()] -> [Split dataset and convert to segments] start converting
0it [00:00, ?it/s]d:\codes\TrajMM\fedtraj\utils\trajclus.py:19: RuntimeWarning: invalid value encountered in double_scalars
  projection = line[0] + (dot_product / line_length_squared) * line_vector
163it [00:00, 514.92it/s]
164it [00:00, 530.72it/s]
164it [00:01, 157.73it/s]
164it [00:00, 217.33it/s]
164it [00:15, 10.72it/s]
[utils.py:128 get_federated_segments()] -> [Split dataset and convert to segments] done


In [7]:
def save_to_pickle(df, file_path, batch_size=16):
    selected_df = df
    merc_seqs = selected_df["merc_seq"].tolist()
    embs = []
    # 按批量大小处理数据
    num_batches = len(merc_seqs) // batch_size + (1 if len(merc_seqs) % batch_size != 0 else 0)
    for i in tqdm(range(num_batches)):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(merc_seqs))
        batch_merc_seqs = merc_seqs[start_idx:end_idx]
        # 批量获取嵌入向量
        batch_embs = batch_get_embedding(batch_merc_seqs)
        embs.append(batch_embs)
    # 拼接所有批次的嵌入向量
    embs = torch.cat(embs, dim=0)
    # 保存嵌入向量到文件
    torch.save(embs, os.path.join(file_path, "embs.pt"))
    # 保存数据框到 pickle 文件
    selected_df.to_pickle(os.path.join(file_path, "segments.pkl"))

In [8]:
for i, dataset in enumerate(traj_dataset):
    save_path = os.path.join(r"D:\codes\TrajMM\resource\results\test", f"fed_{i+1}")
    os.makedirs(save_path, exist_ok=True)
    save_to_pickle(
        dataset.data, save_path, batch_size=16
    )

  0%|          | 0/131 [00:00<?, ?it/s]d:\installation\conda\envs\py310\lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 169/169 [00:25<00:00,  6.60it/s]


In [ ]:
for i, dataset in enumerate(traj_dataset):
    save_path = os.path.join(r"D:\codes\TrajMM\resource\results\test", f"fed_{i+1}")
    os.makedirs(save_path, exist_ok=True)
    save_to_pickle(
        dataset.data, save_path, batch_size=16
    )